Data Pre-processing

In [18]:
#Removed weekends
#got the average for each hour 0-1 [Mon-fri]=B_01, 1-2 [Mon-fri]=B_12, 2-3 [Mon-fri]=B_23 ... 23-24 [Mon-fri]=B2324
#Now the data points, Lets say in 0-1 [Mon-fri] > 0.95*B_01 not an anomaly, else anomaly.  

from statistics import mean
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from pydot import Dot, Edge, Node
from automata.fa.dfa import DFA
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from distances import  custom_pd
import itertools
from time import time

df=pd.read_pickle("C:/Users/bchan/PycharmProjects/test/CaseStudy2021/data/power_demand/labeled/whole/power_data.pkl")

#To start from 1st day of weekday and end on last friday and also to make it multiple of 96 
df=df[480:34560]
c0 = [e[0] for e in df]  
c1 = [e[1] for e in df]

#cwd0 contains data point for weekdays only
df_weekday=[]
weekday_size=480
cwd0=[]
for f in range(0,len(c0),672):
    cwd0.extend(c0[f:f+480])

# Create empty lists for abc and t values
abc = [[] for _ in range(24)]
t = [0] * 24
label = []

# abc contains power consumption of each hour eg: abc[0]-->00am-1am
for j in range(0, len(cwd0), 96):
    for i in range(24):
        abc[i].extend(cwd0[j + i * 4:j + (i + 1) * 4])

# t contains average power consumption of each hour
for i in range(24):
    t[i] = round(mean(abc[i]), 1)

# Calculate label values
for j in range(0, len(cwd0), 96):
    for i in range(24):
        for k in range(4):
            if cwd0[j + i * 4 + k] > t[i] * 0.90 and cwd0[j + i * 4 + k] < t[i] * 1.4:
                label.append(0)
            else:
                label.append(1)

df_day=[]
window_size=96
for f in range(0,len(cwd0),window_size):
    pow=[round(sum(cwd0[j:j+window_size]),1) for j in range(f,f+(window_size),window_size)]
    label_sum=[round(sum(label[j:j+window_size]),1) for j in range(f,f+(window_size),window_size)]
    for i in label_sum:
        # number of single point anomaly of 96 to be allowed
        label1=[0] if i <= 20 else [1]    
    df_day.append(pow + label1)

#split data into train and test
#df_day contains power consumption along with labels, power_day contains power values only 
df_day_train, df_day_test = train_test_split(df_day ,random_state=None, test_size=0.2, shuffle=True)

sorted_data=sorted(set(e[0] for e in df_day))
train_data=[e[0] for e in df_day_train]
train_label=[e[1] for e in df_day_train]
print(f'no of anomalies in train: {sum(train_label)}')

test_data=[e[0] for e in df_day_test]
test_label=[e[1] for e in df_day_test]
print(f'no of anomalies in test: {sum(test_label)}')

def my_scaler(column,min_val,max_val):
    range_val = sorted_data[-1]-sorted_data[0]
    scaled_values = [round(((max_val - min_val)/range_val)*(i - sorted_data[0]) + min_val,1) for i in column]
    return scaled_values

scaled_values=my_scaler(train_data,0,1)
scaled_values1=my_scaler(test_data,0,1)

converted_num = [str(i) for i in scaled_values]
converted_num1 = [str(i) for i in scaled_values1]
Lst=converted_num
Lst1=converted_num1
FL = [l.split(',') for l in Lst]
FL_test = [l.split(',') for l in Lst1]

uniq_list=set(Lst)
alphabet = set(item for string in uniq_list for item in string.split(",") if item != ',')

Pref_S = set()
for string in uniq_list:
    prefixes = string.split(',')
    for i in range(1, len(prefixes) + 1):
        prefix = ','.join(prefixes[:i])
        Pref_S.add(prefix)
Pref_S.add('')

#compute the distance matrix
dist = custom_pd(FL)

no of anomalies in train: 24
no of anomalies in test: 5
(204, 204)


In [7]:
def train(n, eps, eps1, eps2):

    #make changes after testing
    #alphabet, Pref_S, Lst, FL, dist, df_day_test, my_scaler = data_preprocessW()
    #alphabet, Pref_S, Lst, FL, dist, FL_test = data_preprocessW1()
    #alphabet, Pref_S, Lst, FL, dist, FL_test = data_preprocessW()
    
    states = {str(f'q{i}') for i in range(n)}
    start_state = 'q0'

    env=gp.Env(empty=True)
    env.setParam('OutputFlag', 0)
    env.start()
    
    #t0 = time()
    # Creating a new model
    mpdad = gp.Model("DFA_PDAD", env=env)

    t0 = time()

    #DECISION VARIABLES
    delta = mpdad.addVars(states, alphabet, states, vtype=gp.GRB.BINARY, name='delta')
    x = mpdad.addVars(Pref_S, states, vtype=gp.GRB.BINARY, name='x')
    f = mpdad.addVars(states, vtype=gp.GRB.BINARY, name='f')
    alpha = mpdad.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'alpha')
    beta = mpdad.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'beta')

    #OBJECTIVE
    lambda_nn = len(Lst)*(len(Lst)-1)*np.max(dist)
    print(f'eps: {eps}, eps1: {eps1}, eps2: {eps2}')
    mpdad.setObjective(sum(beta[i,state1]*beta[k, state2]*dist[i,k]*(eps/lambda_nn) for i,_ in enumerate(Lst) for state1 in states for k,_ in enumerate(Lst) for state2 in states if dist[i,k] != 0) \
                    + sum(alpha[k,state2]*(eps1/len(Lst)) for k,_ in enumerate(Lst) for state2 in states) \
                    + sum(delta[state1,symbol,state2]*eps2 for state1 in states for symbol in alphabet for state2 in states if state1 != state2),\
                     gp.GRB.MINIMIZE)
    
    #AUTOMATA CONSTRAINTS
    #Constraint1
    for state0 in states:
        for symbol in alphabet:
            mpdad.addConstr(sum(delta[state0,symbol,state1] for state1 in states)==1, name=f'delta[{state0},{symbol}]')

    #Constraint2
    for word in Pref_S:
        mpdad.addConstr(sum(x[word,state1] for state1 in states)==1, name=f'x[{word}]')

    #Constraint3
    mpdad.addConstr(x['',start_state]==1, name='initial_state')

    #Constraint4 
    for state0, word, symbol, state1 in itertools.product(states, Pref_S, alphabet, states):
        if (word + ',' + symbol) in Pref_S:
            mpdad.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[word + ',' + symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')
        if word == '' and symbol in Pref_S:
            mpdad.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')


    #BOUND CONSTRAINTS
    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(alpha[i, state1] >= x[word,state1] + f[state1] -1, name=f'bound_1[{state1},{i}]')        

    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(alpha[i, state1] <= x[word,state1], name=f'bound_2[{state1},{i}]')

    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(alpha[i, state1] <= f[state1], name=f'bound_3[{state1},{i}]')
    
    #not valid MILP constraint
    '''
    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(x[word, state1] * (1-f[state1]) == beta[i, state1], name=f'bound_4[{state1},{i}]')
    '''
    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(beta[i, state1] >= x[word,state1] + (1-f[state1]) -1, name=f'bound_5[{state1},{i}]')

    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(beta[i, state1] <= x[word,state1], name=f'bound_6[{state1},{i}]')

    for i, word in enumerate(Lst):
        for state1 in states:
            mpdad.addConstr(beta[i, state1] <= (1-f[state1]), name=f'bound_7[{state1},{i}]')
    
    mpdad.optimize()
    t1 = time()
    print("Run time", (t1-t0), "seconds")
    mpdad.write(rf'C:\Users\bchan\Desktop\TUD\Thesis\model_PDAD_{n}.lp')

    #print('Obj Value: %g' % mpdad.ObjVal)
    #print(f"Alpha: {mpdad.getAttr('X', alpha)}")
    #print(f"Beta: {mpdad.getAttr('X', beta)}")

    if mpdad.status == 1:
        status = 'LOADED'
        print(f'DFAmodel_{n} LOADED')
    
    elif mpdad.status == 2:
        print(f'DFAmodel_{n} OPTIMAL')
        status='OPTIMAL'
        transitions = mpdad.getAttr('X', delta)
        t_values = [(s1,a,s2) for s1 in states for s2 in states for a in alphabet if round(transitions[s1, a, s2],0) == 1]
        #for t in t_values:
            #print(t)
        f_s = mpdad.getAttr('X', f)
        final_state = {s1 for s1 in states if round(f_s[s1],0) == 1}

        transition_dict = create_transition_dict(states, alphabet, t_values)
        #print(transition_dict)

        dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states=final_state)
        accepted = 0
        rejected = 0
        for w in FL:
            if dfa1.accepts_input(w):
                #print(f'{w}:accepted')
                accepted += 1             
            else:
                #print(f'{w}:rejected')
                rejected += 1        
        print(f'Accepted in Training:{accepted}')
        print(f'Rejected in Training:{rejected}')

        create_diagram(rf'C:\Users\bchan\Desktop\TUD\Thesis\diagram_PDAD_{n}.png', states, start_state,final_state, transition_dict)
        return dfa1, FL_test       
    
    elif mpdad.status == 3:
        status = 'INFEASIBLE'
        print(f'DFAmodel_{n} INFEASIBLE')
    else:
        print('status unknown, DEBUG!!')    
 
    return status

def create_transition_dict(states, alphabet, t_values):
    transition_dict = {}

    for state in states:
        transition_dict[state] = {}
        for symbol in alphabet:
            transition_dict[state][symbol] = None

    for trans in t_values:
        current_state, symbol, next_state = trans
        transition_dict[current_state][symbol] = next_state

    return transition_dict

def create_diagram(path, states, start_state, final_state, transition_dict):

    graph = Dot(graph_type='digraph', rankdir='LR')
    nodes = {}
    for state in states:
        if state == start_state:
            # color start state with green
            if state in final_state:
                initial_state_node = Node(
                    state,
                    style='filled',
                    peripheries=2,
                    fillcolor='#66cc33')
            else:
                initial_state_node = Node(
                    state, style='filled', fillcolor='#66cc33')
            nodes[state] = initial_state_node
            graph.add_node(initial_state_node)
        else:
            if state in final_state:
                state_node = Node(state, peripheries=2)
            else:
                state_node = Node(state)
            nodes[state] = state_node
            graph.add_node(state_node)
    # adding edges
    for from_state, lookup in transition_dict.items():
        for to_label, to_state in lookup.items():
            graph.add_edge(Edge(
                nodes[from_state],
                nodes[to_state],
                label=to_label
            ))
    if path:
        graph.write_png(path)
    return graph

def test(dfa1, FL_test, test_label):
    
    accepted = 0
    rejected = 0
    Predicted_labels=[]
    for w in FL_test:
        #print(w)
        if dfa1.accepts_input(w):
            Predicted_labels.append(1)
            #print('accepted')
            accepted += 1             
        else:
            #print('rejected')
            Predicted_labels.append(0)
            rejected += 1  
              
    print(f'Accepted in Testing:{accepted}')
    print(f'Rejected in Testing:{rejected}')
    accuracy = accuracy_score(test_label, Predicted_labels)
    print(f'Accuracy:{round(accuracy,2)}')
    f1 = f1_score(test_label, Predicted_labels, average='binary', pos_label=1)
    print(f'F1_score:{round(f1,2)}')


In [ ]:
# n denotes the number of states
# eps, eps1, eps2 are regularization parameter
for n in range(2,3):
    for i, j, k in itertools.product([2,3,4], [1], [0]):
        dfa1, FL_test = train(n, eps=i, eps1=j, eps2=k)
        test(dfa1, FL_test, test_label)
